In [115]:
import logging
from time import time
from os import listdir
from numpy.random import RandomState
import matplotlib.pyplot as plt
from cmocean import cm
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from sklearn.cluster import MiniBatchKMeans
from sklearn import decomposition
import cv2
import matplotlib.pylab as pylab
import pandas as pd
import datetime
from datetime import timedelta

%matplotlib inline

In [116]:
directory = '/home/nilavro/data/f03/2017-12-19/1513664059443'
start =  np.float64(sorted(listdir(directory))[0]) 
stop = np.float64(sorted(listdir(directory))[len(sorted(listdir(directory)))-1])

In [117]:
nest = '/home/nilavro/WorkStation/data/Data/NEST/Data_MyHome'

NEST = pd.read_csv(nest + '/nest.csv' ,sep=',')

NEST.tail()

,Unnamed: 0,uname,device_name,timestamp,mode,fan,temperature,humidity,target,away_heat,away_cool,hvac_on,online
2158,2158,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,73.0,0.35,73.0,50.0,90.0,hea,1
2159,2159,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,hea,1
2160,2160,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,off,1
2161,2161,nilavra,Dining Room Thermostat,1.513994e+09,heat,0,73.0,0.35,73.0,50.0,90.0,off,1
2162,2162,nilavra,Dining Room Thermostat,1.513995e+09,heat,0,73.0,0.35,73.0,50.0,90.0,off,1


In [118]:
NEST.loc[NEST.hvac_on.isin(['off']), 'hvac_on'] = 0
NEST.loc[NEST.hvac_on.isin(['hea']), 'hvac_on'] = 1
NEST.loc[NEST.hvac_on.isin(['coo']), 'hvac_on'] = -1


,Unnamed: 0,uname,device_name,timestamp,mode,fan,temperature,humidity,target,away_heat,away_cool,hvac_on,online
2158,2158,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,73.0,0.35,73.0,50.0,90.0,1,1
2159,2159,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,1,1
2160,2160,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,0,1
2161,2161,nilavra,Dining Room Thermostat,1.513994e+09,heat,0,73.0,0.35,73.0,50.0,90.0,0,1
2162,2162,nilavra,Dining Room Thermostat,1.513995e+09,heat,0,73.0,0.35,73.0,50.0,90.0,0,1


In [163]:
NEST['date'] = pd.to_datetime(NEST['timestamp'], unit='s' ) - timedelta(hours=5 )
NEST.tail()


,Unnamed: 0,uname,device_name,timestamp,mode,fan,temperature,humidity,target,away_heat,away_cool,hvac_on,online,date
2158,2158,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,73.0,0.35,73.0,50.0,90.0,1,1,2017-12-22 20:30:13.517
2159,2159,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,1,1,2017-12-22 20:36:23.337
2160,2160,nilavra,Dining Room Thermostat,1.513993e+09,heat,0,74.0,0.35,73.0,50.0,90.0,0,1,2017-12-22 20:38:18.025
2161,2161,nilavra,Dining Room Thermostat,1.513994e+09,heat,0,73.0,0.35,73.0,50.0,90.0,0,1,2017-12-22 20:54:16.504
2162,2162,nilavra,Dining Room Thermostat,1.513995e+09,heat,0,73.0,0.35,73.0,50.0,90.0,0,1,2017-12-22 21:06:18.740


In [ ]:
def round_minutes(dt, direction, resolution):
    new_minute = (dt.minute // resolution + (1 if direction == 'up' else 0)) * resolution
    return dt + datetime.timedelta(minutes=new_minute - dt.minute)

In [ ]:
NEST['date'] = pd.to_datetime(NEST['timestamp'], unit='s' ) - timedelta(hours=5 )


In [154]:
NESTED = pd.Series(pd.date_range('2017-12-06', '2017-12-23', freq='min')).to_frame()
NESTED.columns = ['DateTime']
NESTED['HVAC']  = np.nan
NESTED.head()

,DateTime,HVAC
0,2017-12-06 00:00:00,NaN
1,2017-12-06 00:01:00,NaN
2,2017-12-06 00:02:00,NaN
3,2017-12-06 00:03:00,NaN
4,2017-12-06 00:04:00,NaN


In [ ]:
NEST['date'] = 